In [1]:
from EDTpy.ChipElements import *
import numpy as np
import copy
from EDTpy import settings
import EDTpy
from EDTpy import EmptyGeometry
from EDTpy import EmptyPath
from EDTpy.settings import *
import gdspy

# setting cpw parameters
CPW.default_values['S'] = 20
CPW.default_values['W'] = 12
CPW.default_values['layer'] = 0

dc_s = 2
dc_w = 2

In [2]:
class Marker(EmptyGeometry):
    default_values = {
        "a": 20,
        "b": 100,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "Marker"
        
        a = values['a']
        b = values['b']
        layer = values['layer']
        
        self + gdspy.Rectangle([-b/2-b/10, -b/2-b/10],
                               [b/2+b/10, b/2+b/10], layer = 0)
        
        mark = gdspy.boolean(gdspy.Rectangle([-b/2, -a/2],[b/2, a/2]), gdspy.Rectangle([-a/2, -b/2],
                               [a/2, b/2]), 'or', layer = 0)
        self - mark
        self.add_port([0, 0], 0)

In [23]:
class res(EmptyGeometry):
    default_values = {
        "s": CPW.default_values['S'],
        "w": CPW.default_values['W'],
        "l": 5000,
        "layer": 0,   
    }
    
    def _drawing(self, values):
        self.name = "GraphenHole"
        
        self.add_port([0, 0], -90) #0


In [24]:
sketch = EDTpy.EmptySketch()
test = res()
sketch.add_geometry(test)
test.ports[0].position
test.show()

In [17]:
class GraphenHole(EmptyGeometry):
    default_values = {
        "size1": 790,
        "size2": 280,
        "layer": 0,
        "contact_S": 2,
        "contact_b": 12,
        "contact_L": 30,        
    }

    def _drawing(self, values):
        self.name = "GraphenHole"
        
        self + gdspy.Rectangle([-values['size1']/2, -values['size2']/2],
                               [values['size1']/2, values['size2']/2],
                               layer = values['layer'])
        
        width1 = 95
        h1 = values['size2']/2
        
        width2 = 25
        h2 = h1+15
        
        width3 = 7.5
        h3 = h1+30
        
        h4 = h1+40
        points = [(width1, h1), 
                  (width2, h2), (width3, h3), (width3, h4), (-width3, h4),(-width3, h3),(-width2, h2), 
                  (-width1, h1)]
        self + gdspy.Polygon(points)
        
        self - gdspy.Rectangle([-2.5, h4],
                               [2.5, h4-30],
                               layer = values['layer'])
        
        self - gdspy.Rectangle([-width2, h2-5],
                               [-width2+5, h2+5],
                               layer = values['layer'])
        
        self - gdspy.Rectangle([width2, h2-5],
                               [width2-5, h2+5],
                               layer = values['layer'])
        
        self.add_port([0, h4], 90) #0
        
        self + gdspy.Rectangle([-30, -h1],
                               [30, -h1-70],
                               layer = values['layer'])
        
        self - gdspy.Rectangle([-20, -h1+45],
                               [-18, -h1-70],
                               layer = values['layer'])
        self.add_port([-19, -h1-70], -90) #1
        
        self - gdspy.Rectangle([20, -h1+45],
                               [18, -h1-70],
                               layer = values['layer'])
        self.add_port([19, -h1-70], -90) #2
        
        
        self - gdspy.Rectangle([-6, -h1+45],
                               [-8, -h1-70],
                               layer = values['layer'])
        self.add_port([-7, -h1-70], -90) #3
        
        
        self - gdspy.Rectangle([6, -h1+45],
                               [8, -h1-70],
                               layer = values['layer'])
        self.add_port([7, -h1-70], -90) #4

In [14]:
class CPW_C(EmptyGeometry):
    default_values = {
        "s": CPW.default_values['S'],
        "w": CPW.default_values['W'],
        "N": 2, #number of odd elements
        "l": 50, 
        "a": 3.3, # width of finger
        "b": 3.3, # gap between fingers
        "g": 15, # gap between fingers and GND
        "contact_pad_size": 300,
        "layer": 0,
    }

    def _drawing(self, values):
        self.name = "CPW_C"
        
        w = values['w']
        s = values['s']
        N = int((values['N'])/2)
        a = values['a']
        b = values['b']
        l = values['l']+b*2
        g = values['g']
        gnd = 30
        
        contact_pad_size = values['contact_pad_size']
        layer = values['layer']
        
        self + gdspy.Rectangle([0, -s/2-w],
                               [50, s/2+w], layer = 0)
        self - gdspy.Rectangle([0, -s/2],
                               [50, s/2], layer = 0)
            
        L = l+gnd
        jo = s/2
        jo_g = s/2+w


        
        z = 50
        self + gdspy.Rectangle([z+100, -contact_pad_size/2-50],
                               [z+100+contact_pad_size+50, contact_pad_size/2+50], layer = 0)
        self - gdspy.Rectangle([z+100, -contact_pad_size/2],
                       [z+100+contact_pad_size, contact_pad_size/2], layer = 0)
        
        points = [(z, jo_g), 
                  (z+100, contact_pad_size/2+50), (z+100, contact_pad_size/2), 
                  (z, jo)]
        self + gdspy.Polygon(points)
        
        points = [(z, -jo_g), 
                  (z+100, -contact_pad_size/2-50), (z+100, -contact_pad_size/2), 
                  (z, -jo)]
        self + gdspy.Polygon(points)

        self.add_port([0, 0], 180)

In [16]:
sketch = EDTpy.EmptySketch()
sketch + gdspy.Rectangle([-5000/2, -5000/2],
                         [5000/2, 5000/2], layer = 0)

sketch - gdspy.Rectangle([-5000/2 + 250, -5000/2 + 250],
                         [5000/2 - 250, 5000/2 - 250], layer = 0)

sketch.add_port([-2000, 2000], -180) #Marker port  --  0

marker = Marker()
marker.merge_with(sketch.ports[0], 0)
sketch.add_geometry(marker)
sketch.circular_array(marker)

sketch.add_port([-1700, 0], 180) #Marker port  --  1
sketch.add_port([1700, 0], 0) #Marker port  --  2

CPW_1 = CPW_C()
CPW_1.merge_with(sketch.ports[1], 0)
sketch.add_geometry(CPW_1)

CPW_2 = CPW_C()
CPW_2.merge_with(sketch.ports[2], 0)
sketch.add_geometry(CPW_2)

h = 700
res_path = [sketch.ports[1].position, 
            sketch.ports[2].position]
res_R = 150
res = CPW(res_path, res_R)
print(res.length)
sketch.add_geometry(res)


sketch.assemble()
sketch.save_gds("C:/Users/mukha/Documents/Aalto/Graphen/Munich 2.0/Notch")
# sketch.show()



Assembling Empty Sketch:   0%|                                                                   | 0/7 [00:00<?, ?it/s]

3400.0
